In [1]:
print("Bismillah Hirrahamaa Nirraheem")

Bismillah Hirrahamaa Nirraheem


In [2]:
import os,sys,warnings,time,re,math
warnings.filterwarnings("ignore")
from IPython.display import clear_output
os.cpu_count()

12

In [3]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
from typing import Literal
from multiprocessing import Process
from concurrent.futures import ThreadPoolExecutor,ProcessPoolExecutor,wait
pio.templates.default = "plotly_dark"
import keras
from keras import ops
import tensorflow as tf
from tensorflow.io import TFRecordWriter,TFRecordOptions,FixedLenFeature,VarLenFeature,serialize_tensor,parse_tensor,parse_single_example
from tensorflow.data import TFRecordDataset,Dataset
from tensorflow.train import Feature,Features,BytesList,FloatList,Int64List,Example
from sklearn.datasets import fetch_openml
from google.cloud import storage
client = storage.Client("kaggle-406814")
tf.get_logger().setLevel("ERROR")
from contextlib2 import ExitStack
from object_detection.dataset_tools import tf_record_creation_util
from tqdm import tqdm
from zipfile import ZipFile
%matplotlib inline

# Dataset Loading

In [4]:
PATH = "gs://stanfordrna/mnist/*.tfrecord"
all_files = tf.io.gfile.glob(PATH)
train_files = all_files[:15]
valid_files = all_files[15:18]
test_files = all_files[18:]
train_raw = TFRecordDataset(train_files,compression_type="GZIP")
test_raw = TFRecordDataset(test_files,compression_type="GZIP")
valid_raw = TFRecordDataset(valid_files,compression_type="GZIP")

In [5]:
mnist_feature = dict(
    image=VarLenFeature(tf.string),
    label = FixedLenFeature([],tf.float32)
)

def mnist_example(example):
    example = parse_single_example(example,mnist_feature)
    example["image"] = parse_tensor(tf.sparse.to_dense(example["image"])[0],out_type=tf.float32)
    return example

def create_ds(ds:tf.data.Dataset,batch_size:int,is_train:bool=False,shuffle_size:int=5000):
    ds = ds.map(mnist_example,num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.map(lambda example: (example["image"],example["label"]),num_parallel_calls=tf.data.AUTOTUNE)
    if is_train:
        ds = ds.shuffle(shuffle_size)
        ds = ds.repeat()
    ds = ds.batch(batch_size,drop_remainder=True)
    if not is_train:
        ds = ds.cache()
    return ds.prefetch(tf.data.AUTOTUNE)


BATCH_SIZE = 32
train_ds = create_ds(train_raw,batch_size=BATCH_SIZE,is_train=True)
valid_ds = create_ds(valid_raw,batch_size=BATCH_SIZE,is_train=False)
test_ds = create_ds(test_raw,batch_size=BATCH_SIZE,is_train=False)

In [7]:
X,y = train_ds.take(1).get_single_element()
print("X shape: ",X.shape)
print("y shape: ",y.shape)

DataLossError: {{function_node __wrapped__DatasetToSingleElement_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} corrupted record at 1299252 [Op:DatasetToSingleElement] name: 